In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_qdrant import QdrantVectorStore
from langchain_ollama import OllamaEmbeddings
from langchain_qdrant import FastEmbedSparse, RetrievalMode
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.load import dumps, loads
from typing_extensions import List
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
from langchain.retrievers.document_compressors import FlashrankRerank

import config

# Setup

In [2]:
url = config.QDRANT_URL
api_key = config.QDRANT_API_KEY

embeddings = OllamaEmbeddings(
    model="nomic-embed-text:v1.5"
)

sparse_embeddings = FastEmbedSparse(
    model_name="Qdrant/bm25"
)

qdrant = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    sparse_embedding=sparse_embeddings,
    url=url,
    prefer_grpc=True,
    api_key=api_key,
    collection_name="mlops_document",
    retrieval_mode=RetrievalMode.HYBRID,
)

qdrant_retriever = qdrant.as_retriever(
    search_type="similarity", 
    search_kwargs={"k": 5},
)

groq_llama3_1_70b = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0,
    api_key=config.GROQ_API_KEY
)


Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

# 1. Retrieval using RAG Fusion

In [3]:
def get_top_docs(retrieved_docs: list[list], top_k:int, question:str):

    flattened_docs = [doc for docs in retrieved_docs for doc in docs]

    flash_reranker = FlashrankRerank(
        model="ms-marco-MiniLM-L-12-v2",
        top_n=top_k
    )

    top_docs = flash_reranker.compress_documents(
        documents=flattened_docs, query=question
    )

    return top_docs

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


def rag_fusion(
    query: str
) -> str:
    # ----------------------------------------------------------------
    # 1. Query Generation
    # ----------------------------------------------------------------
    # RAG-Fusion: Related
    template = """
    You are an intelligent assistant that specializes in breaking down complex queries into simpler, manageable subqueries. 
    Generate exactly 4 focused search queries that each address a unique aspect of the following question: {question}
    Provide only the search queries as individual questions, and do not include any additional text or explanations.
    """

    prompt_rag_fusion = ChatPromptTemplate.from_template(template)

    llm = groq_llama3_1_70b

    generate_queries = (
        prompt_rag_fusion 
        | llm
        | StrOutputParser() 
        | (lambda x: x.split("\n"))
    )

    # ----------------------------------------------------------------
    # 2. Retrieval
    # ----------------------------------------------------------------
    retrieval_chain_rag_fusion = generate_queries | qdrant_retriever.map() 

    retrieved_docs = retrieval_chain_rag_fusion.invoke({"question": query})

    top_docs = get_top_docs(retrieved_docs=retrieved_docs, top_k=4, question=query)

    doc_context = format_docs(top_docs)

    # ----------------------------------------------------------------
    # 3. Generation
    # ----------------------------------------------------------------
    # Prompt
    template = """
    You are an assistant for question-answering tasks. 
    Please answer the question based on the context provided.
    Do not tell the user that you are referring to the context to answer the question 
    If you don't know the answer or the context does not answer the question, just say that you don't know. 
    Elaborate your answer in well structured format.

    Question: {question} 
    Context: {context} 
    Answer:
    """

    prompt = ChatPromptTemplate.from_template(template)

    generate_llm = groq_llama3_1_70b

    final_rag_chain = (
        prompt
        | generate_llm
        | StrOutputParser()
    )

    # Return query result
    return final_rag_chain.invoke({"question": query, "context":doc_context}), top_docs, doc_context

# 2. Test and evaluate

## 2.1 Test the function

In [4]:
question = "what is data distribution drift?"
result, refer_docs, doc_context = rag_fusion(question)
print(result)

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Data distribution drift, also known as data distribution shift, refers to the change in the underlying distribution of the data over time. This can occur in various forms, including changes in the input data (covariate shift), changes in the output data (label shift), or changes in the relationship between the input and output data (concept drift).

There are three distinct subtypes of data distribution drift:

1. **Covariate Shift**: This occurs when the probability density of the input data (P(X)) changes, but the conditional probability of the output given the input (P(Y|X)) remains the same.

2. **Label Shift**: This occurs when the probability density of the output data (P(Y)) changes, but the conditional probability of the input given the output (P(X|Y)) remains the same.

3. **Concept Drift**: This occurs when the conditional probability of the output given the input (P(Y|X)) changes, but the probability density of the input data (P(X)) remains the same.

Data distribution drift

## 2.2 Generate answer and retrieve the referred context

In [5]:
import pandas as pd
import joblib

rag_fusion_test = joblib.load(filename="test_cases.pkl")

INFO:numexpr.utils:NumExpr defaulting to 12 threads.


In [6]:
from tqdm import tqdm
import time

def answer_question_llm(
    test_case: dict,
    question_key: str,
    answer_key: str,
    doc_context_key: str
) -> dict:
    rag_fusion_test[answer_key] = []
    rag_fusion_test[doc_context_key] = []

    for question in tqdm(test_case[question_key], desc="Answering questions", total=len(test_case[question_key])):
        result, retrieved_docs, doc_context = rag_fusion(question)
        test_case[answer_key].append(result)
        test_case[doc_context_key].append(doc_context)

        time.sleep(10) # Stop a bit, dont spam the API
    return test_case


In [7]:
rag_fusion_test = answer_question_llm(
    test_case=rag_fusion_test,
    question_key="question",
    answer_key="rag_fusion_answer",
    doc_context_key="rag_fusion_doc_context"
)

Answering questions:   0%|          | 0/20 [00:00<?, ?it/s]INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
Answering questions:   5%|▌         | 1/20 [00:15<05:03, 15.96s/it]INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://127.0.0.1:11434/api/embed "HTTP/1.1 200 OK"
INFO:httpx:HTTP Req

In [8]:
print(rag_fusion_test['rag_fusion_answer'][0])

**Data Distribution Shifts: Definition and Explanation**

Data distribution shifts refer to changes in the underlying data distribution of a machine learning model. This can occur in various forms, including changes in the input data, output data, or the relationship between the input and output data.

**Types of Data Distribution Shifts**

There are three distinct subtypes of data shifts:

1. **Covariate Shift**: This occurs when the probability density of the input data (P(X)) changes, but the conditional probability of the output given the input (P(Y|X)) remains the same.
2. **Label Shift**: This occurs when the probability density of the output data (P(Y)) changes, but the conditional probability of the input given the output (P(X|Y)) remains the same.
3. **Concept Drift**: This occurs when the conditional probability of the output given the input (P(Y|X)) changes, but the probability density of the input data (P(X)) remains the same.

**Causes and Consequences of Data Distribution

## 2.3 Deepeval RAG Evaluation

In [21]:
from langchain_groq import ChatGroq
from deepeval.models.base_model import DeepEvalBaseLLM
from deepeval.metrics import ContextualPrecisionMetric, ContextualRecallMetric, ContextualRelevancyMetric, AnswerRelevancyMetric, FaithfulnessMetric
from deepeval.test_case import LLMTestCase
from tqdm import trange

class GroqLLM(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        return chat_model.invoke(prompt).content

    async def a_generate(self, prompt: str) -> str:
        chat_model = self.load_model()
        res = await chat_model.ainvoke(prompt)
        return res.content

    def get_model_name(self):
        return "Custom Azure OpenAI Model"

# Replace these with real values
custom_model = ChatGroq(
    model="gemma-7b-it",
    temperature=0,
    api_key=config.GROQ_API_KEY
)
# Define Critic model
groq_critic_llm = GroqLLM(model=custom_model)

# Define all metrics
contextual_precision = ContextualPrecisionMetric(
    threshold=0.7,
    model=groq_critic_llm,
    include_reason=False
)

contextual_recall = ContextualRecallMetric(
    threshold=0.7,
    model=groq_critic_llm,
    include_reason=False    
)
contextual_relevancy = ContextualRelevancyMetric(
    threshold=0.7,
    model=groq_critic_llm,
    include_reason=False
)

answer_relevancy = AnswerRelevancyMetric(
    threshold=0.7,
    model=groq_critic_llm,
    include_reason=False
)

faithfulness = FaithfulnessMetric(
    threshold=0.7,
    model=groq_critic_llm,
    include_reason=False
)


def evaluate_rag(
    rag_test_cases: dict,
    answer_relevancy_key: str,
    faithfulness_key: str,
    contextual_precision_key: str,
    contextual_recall_key: str,
    contextual_relevancy_key: str,
    question_key: str,
    expected_answer_key: str,
    actual_answer_key: str,
    doc_context_key: str
) -> dict:

    rag_test_cases[answer_relevancy_key] = []
    rag_test_cases[faithfulness_key] = []
    rag_test_cases[contextual_precision_key] = []
    rag_test_cases[contextual_recall_key] = []
    rag_test_cases[contextual_relevancy_key] = []

    # Evaluate each test case
    for i in tqdm(range(len(rag_test_cases[question_key])), desc="Evaluating answers", total=len(rag_test_cases[question_key])):
        question = rag_test_cases[question_key][i]
        expected_answer = rag_test_cases[expected_answer_key][i]
        actual_answer = rag_test_cases[actual_answer_key][i]
        doc_context = rag_test_cases[doc_context_key][i]


        test_case = LLMTestCase(
            input=question,
            actual_output=actual_answer,
            expected_output=expected_answer,
            retrieval_context=[
                doc_context
            ]
        )

        try:
            answer_relevancy.measure(test_case)
            answer_relevancy_score = answer_relevancy.score
        except:
            answer_relevancy_score = -1
        time.sleep(20)

        try:
            faithfulness.measure(test_case)
            faithfulness_score = faithfulness.score
        except:
            faithfulness_score = -1
        time.sleep(20)   

        try:
            contextual_precision.measure(test_case)
            contextual_precision_score = contextual_precision.score
        except:
            contextual_precision_score = -1
        time.sleep(20)

        try:
            contextual_recall.measure(test_case)
            contextual_recall_score = contextual_recall.score
        except:
            contextual_recall_score = -1
        time.sleep(20)

        try:
            contextual_relevancy.measure(test_case)
            contextual_relevancy_score = contextual_relevancy.score
        except:
            contextual_relevancy_score = -1
        time.sleep(20)

        rag_test_cases[answer_relevancy_key].append(answer_relevancy_score)
        rag_test_cases[faithfulness_key].append(faithfulness_score)
        rag_test_cases[contextual_precision_key].append(contextual_precision_score)
        rag_test_cases[contextual_recall_key].append(contextual_recall_score)
        rag_test_cases[contextual_relevancy_key].append(contextual_relevancy_score)

        time.sleep(5)

    return rag_test_cases





In [22]:
rag_fusion_test = evaluate_rag(
    rag_test_cases=rag_fusion_test,
    answer_relevancy_key="answer_relevancy",
    faithfulness_key="faithfulness",
    contextual_precision_key="contextual_precision",
    contextual_recall_key="contextual_recall",
    contextual_relevancy_key="contextual_relevancy",
    question_key="question",
    expected_answer_key="expected_answer",
    actual_answer_key="rag_fusion_answer",
    doc_context_key="rag_fusion_doc_context"
)



Evaluating answers:   0%|          | 0/20 [00:00<?, ?it/s]

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


Evaluating answers: 100%|██████████| 20/20 [41:13<00:00, 123.68s/it]


In [24]:
rag_fusion_test.keys()

dict_keys(['question', 'expected_answer', 'query_type', 'rag_fusion_answer', 'rag_fusion_doc_context', 'answer_relevancy', 'faithfulness', 'contextual_precision', 'contextual_recall', 'contextual_relevancy'])

In [23]:
print(rag_fusion_test['rag_fusion_answer'][0])

**Data Distribution Shifts: Definition and Explanation**

Data distribution shifts refer to changes in the underlying data distribution of a machine learning model. This can occur in various forms, including changes in the input data, output data, or the relationship between the input and output data.

**Types of Data Distribution Shifts**

There are three distinct subtypes of data shifts:

1. **Covariate Shift**: This occurs when the probability density of the input data (P(X)) changes, but the conditional probability of the output given the input (P(Y|X)) remains the same.
2. **Label Shift**: This occurs when the probability density of the output data (P(Y)) changes, but the conditional probability of the input given the output (P(X|Y)) remains the same.
3. **Concept Drift**: This occurs when the conditional probability of the output given the input (P(Y|X)) changes, but the probability density of the input data (P(X)) remains the same.

**Causes and Consequences of Data Distribution

In [25]:
import numpy as np

print(f"answer_relevancy: {np.average(rag_fusion_test['answer_relevancy'])}")
print(f"faithfulness: {np.average(rag_fusion_test['faithfulness'])}")
print(f"contextual_precision: {np.average(rag_fusion_test['contextual_precision'])}")
print(f"contextual_recall: {np.average(rag_fusion_test['contextual_recall'])}")
print(f"contextual_relevancy: {np.average(rag_fusion_test['contextual_relevancy'])}")

answer_relevancy: 0.9720238095238095
faithfulness: 0.785357142857143
contextual_precision: 0.3
contextual_recall: -0.8
contextual_relevancy: 0.595297619047619


In [26]:
rag_fusion_test['contextual_relevancy']

[0.8,
 0.6,
 0.8333333333333334,
 0.8333333333333334,
 0.7142857142857143,
 0.8,
 0.6,
 0.4,
 0.875,
 0.6666666666666666,
 0.5,
 0.5,
 0.8,
 0.8,
 0.6666666666666666,
 -1,
 0.75,
 0.5,
 0.6,
 0.6666666666666666]

In [27]:
import joblib
joblib.dump(value=rag_fusion_test, filename="rag_fusion_test.pkl")

['rag_fusion_test.pkl']